# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/workspace/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Apache Cassandra coding portion

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)
    
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

#### Question 1
Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

#### Answer
- For this question, we want to retrieve data based on two filtering conditions: sessionId and itemInSession. 
Therefore, we create a new table called event_by_session_item.

- The primary key is defined as (session_id, itemInSession). 

- This choice allows efficient querying based on these two columns.
Since we need to filter by session_id and itemInSession, we make them the primary key columns. 

- We will also include artist, song title and song length because the question is looking for those information

#### Create Table

In [30]:
query = "CREATE TABLE IF NOT EXISTS event_by_session_item "
query = query + "(session_id int, itemInSession int, artist text, song text, length float, PRIMARY KEY (session_id, itemInSession))"

try:
    session.execute(query)
except Exception as e:
    print(e)


#### Insert data

In [31]:
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader  (f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO event_by_session_item (session_id, itemInSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        ### the index is based on the position of the columns
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Query the data

In [33]:
query = "SELECT artist, song, length FROM event_by_session_item WHERE session_id = 338 AND itemInSession = 4;"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### Question 2
Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182


### Answer 
- For this question, we want to retrieve data based on filtering conditions for user_id and session_id, and we also want the results to be sorted by itemInSession.
- We create a new table called event_by_user_session.
- The primary key is defined as ((user_id, session_id), itemInSession). This primary key allows us to filter data based on user_id and session_id, and since itemInSession is included in the primary key, the results will be sorted by itemInSession.
- We will also include artist, song and user firstname, lastname to provide the full context

#### Create Table

In [34]:
query = "CREATE TABLE IF NOT EXISTS event_by_user_session "
query = query + "(user_id int, session_id int, itemInSession int, artist text, song text, first_name text, last_name text, PRIMARY KEY ((user_id, session_id), itemInSession))"

try:
    session.execute(query)
except Exception as e:
    print(e)

#### Insert Data

In [ ]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO event_by_user_session (user_id, session_id, itemInSession, artist, song, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ### the index is based on the position of the columns
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

#### Query and Print the data

In [37]:
query = "SELECT artist, song, first_name, last_name FROM event_by_user_session WHERE user_id = 10 AND session_id = 182 ORDER BY itemInSession;"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.first_name, row.last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Question 3
Give every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

### Answer 
- For this question, we want to retrieve data based on the filtering condition for the song.
- We create a new table called event_by_song.
- The primary key is defined as (song, user_id). This choice allows us to filter data based on the song, and the user_id is included to make the primary key unique.
- Also, since we are not filtering by session-related attributes in this query, we don't include them in the primary key. This design allows efficient querying based on the song attribute.
- We include user_id, song, first_name, last_name columns to provide the full context


#### Create the table

In [ ]:
query = "CREATE TABLE IF NOT EXISTS event_by_songs "
query = query + "(song text, user_id int, first_name text, last_name text, PRIMARY KEY (song, user_id))"

try:
    session.execute(query)
except Exception as e:
    print(e)      

#### Insert the data

In [ ]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO event_by_songs (song, user_id, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))  

#### Query the data

In [39]:
query = "SELECT first_name, last_name FROM event_by_songs WHERE song = 'All Hands Against His Own';"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.first_name, row.last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [43]:
query = "drop table event_by_session_item"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [ ]:
query = "drop table event_by_user_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [44]:
query = "drop table event_by_songs"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [45]:
session.shutdown()
cluster.shutdown()